In [37]:
import rdflib
import html
from SPARQLWrapper import SPARQLWrapper , JSON, CSV, TSV, TURTLE
import pandas as pd

g = rdflib.Graph().parse('./fb15k-wikitop2021-yago-facts.nt', format='ntriples')
g += rdflib.Graph().parse('./fb15k-wikitop2021-yago-full-types.nt', format='ntriples')

from rdflib.namespace import RDF
for line in open('fb15k-wikitop2021.tsv').readlines()[1:]:
    subject = rdflib.URIRef( line.split('\t')[6][1:-1] ) # take the URI in column 6
    g.add( (subject, RDF.type, rdflib.URIRef('http://example.com/popularEntity')) )

In [38]:
r = g.query("""
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10
""")
for b in r.bindings:
    print(*[f'{x.split("/")[-1][:40]:40s}' for x in b.values()])


22-rdf-syntax-ns#type                    popularEntity                            186                                     
22-rdf-syntax-ns#type                    Human                                    134                                     
knowsLanguage                            English_language                         118                                     
nationality                              United_States                            93                                      
hasOccupation                            Actor                                    77                                      
hasOccupation                            Film_producer                            51                                      
hasOccupation                            Screenwriter                             37                                      
hasOccupation                            Film_director                            32                                      
memberOf        

In [53]:
%load_ext ipython_sparql_pandas
sparql = SPARQLWrapper("http://DESKTOP-D6TQ7G4:7200/repositories/20qs")

The ipython_sparql_pandas extension is already loaded. To reload it, use:
  %reload_ext ipython_sparql_pandas


In [228]:
%%sparql http://DESKTOP-D6TQ7G4:7200/repositories/20qs -s qres 
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10

,count,p,o
0,186,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.com/popularEntity
1,134,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Human
2,118,http://schema.org/knowsLanguage,http://yago-knowledge.org/resource/English_lan...
3,93,http://schema.org/nationality,http://yago-knowledge.org/resource/United_States
4,77,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Actor
5,51,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Film_producer
6,37,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Screenwriter
7,32,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Film_director
8,29,http://schema.org/memberOf,http://yago-knowledge.org/resource/Democratic_...
9,25,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Country


In [238]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
def extractCountAndAttr(result):
    listResults=[]
    res = qres.get('results')
    res = res.get('bindings')
    for i in res:
        for j in i.values():
            value = j.get('value')
            listResults.append(value)      
    listPredicates = listResults[1::3]
    listObjects = listResults[2::3]
    listCounts = listResults[::3]
    listCounts = [eval(i) for i in listCounts]
    
    return(listCounts, listObjects, listPredicates)

#function to approximate index element of list that provides best split
def bestSplit(listOfNumbers, ElementsLeft):
    currentBest=1
    indexBest=-1
    for i in range(len(listOfNumbers)):
        splitScore = abs((listOfNumbers[i]/ElementsLeft)-0.5)
        if splitScore<currentBest:
            currentBest=splitScore
            indexBest=i
    return indexBest


def generateQuestion(listOfPred, listOfObj, index):
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]} ?' )
    if userAnswer.startswith('y'):
        PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.' )   
    elif userAnswer.startswith('n'):
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}' )   
    else:
        generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)


  
def updateQuery( PosFilters, NegFilters):

    FilterQuestion =  f"""
    select (count(*) as ?count) ?p ?o  where 
    {{

    {PosFilters}
     ?s ?p ?o .
    {NegFilters}
    }} 
    group by ?p ?o 
    order by desc(?count)
    limit 25
    """
        
    return FilterQuestion




In [248]:
#reset query, resets program
FilterQuestion =  f"""
    select (count(*) as ?count) ?p ?o  where 
    {{

    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
     ?s ?p ?o .
    }} 
    group by ?p ?o 
    order by desc(?count)
    limit 25
    """
sparql.setQuery(FilterQuestion)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()


PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
index= bestSplit(availableSplits, availableSplits[0])

In [247]:
#play 20qs
i=0
while i< 20:
   
        
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    if availableSplits[0]<=1:
        break
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()

    
finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
finalRes = g.query(finalResult)
for b in finalRes.bindings:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in b.values()][0:10]) 

Does the thing you are looking for have the attribute: nationality United_States ? y
Does the thing you are looking for have the attribute: hasOccupation Film_producer ? n
Does the thing you are looking for have the attribute: hasOccupation Actor ? n
Does the thing you are looking for have the attribute: memberOf Democratic_Party_(United_States) ? y
Does the thing you are looking for have the attribute: award Presidential_Medal_of_Freedom ? n
Does the thing you are looking for have the attribute: alumniOf Columbia_University ? n
Does the thing you are looking for have the attribute: alumniOf Lee_Strasberg_Theatre_and_Film_Institute ? y
Does the thing you are looking for have the attribute: image Scarlett%20Johansson%20by%20Gage%20Skidmore%202%20%28cropped%29%20%28cropped%29.jpg ? y


Your Answer is: Scarlett_Johansson


In [241]:
#Always pritning the posible spilits , doesnt work now
i=0
while i< 20:
    if availableSplits[0]<=1:
        break
        
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    qres = g.query(query)
    for b in qres.bindings:
        print(*[f'{x.split("/")[-1][:40]:40s}' for x in b.values()][0:10]) 

finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
finalRes = g.query(finalResult)
for b in finalRes.bindings:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in b.values()][0:10]) 

Your Answer is: Jennifer_Lawrence


In [ ]:
#Cell To Reset the program: usless now
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
qres = g.query("""
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10
""")
availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
index= bestSplit(availableSplits, availableSplits[0])